In [1]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
import main.packages.unchain_chain as chain
import main.packages.mine_generic as mine_g
import numpy as np
import pandas as pd
import seaborn as sns
import re
from sklearn.metrics import mean_squared_error
from dieboldmariano import dm_test

In [2]:
head_infl = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')
head_infl_new = head_infl.loc[:, ['ar_110_h_1', 'ar_110_h_2', 'ar_110_h_3', 'ar_ols_h_1', 'ar_ols_h_2',
       'ar_ols_h_3']]
head_infl_new.to_csv('data/headline_forecast/head_infl_forecast.csv', index=False)

In [3]:
def take_y_reals():
    cats = ["food", "energy", "neig", "services"]
    
    concatenated_df = pd.DataFrame()

    for cat in cats:
        cat_df = mine_g.load_excel(f"data/hicp_cat_raw/prc_hicp_{cat}.xlsx", name = f"hicp_{cat}", subset=True, verbose=0)
        #cat_df = cat_df[(cat_df.index > mine_g.train_test_split_date)]
        concatenated_df = pd.concat([concatenated_df, cat_df], axis=1)
    return concatenated_df

def extract_forecast_model_h(forecast_all_cat_df, model, h):
    """
    in 4 df, extract columns with model's name and horizon
    """

    pattern = rf"^{model}_(?![lp]).*h_{h}"
    matching_cols = [col for col in forecast_all_cat_df.columns if re.search(pattern, col)]
    df_specific_model_h = forecast_all_cat_df.loc[:, matching_cols]
    return df_specific_model_h

import os

def concatenate_csv_files(folder_path):
    """
    Concatenates CSV files in a folder column-wise into a single DataFrame.

    Args:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - concatenated_df (DataFrame): Concatenated DataFrame.
    """

    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    concatenated_df = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Add each file's data as new columns in the concatenated DataFrame
        concatenated_df = pd.concat([concatenated_df, df], axis=1)
        concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]

    return concatenated_df

def transform_to_chained(y_unchain, y_real):
    y_real = y_real[y_real.index <= "2022-12-31"]
    dec_mask = y_real.index.month == 12
    dec_data = y_real.where(dec_mask, other=np.nan)
    dec_data.ffill(inplace=True)
    dec_data = dec_data[dec_data.index > "2015-12-31"]
    # chain data back:
    y_chained = y_unchain * dec_data
    return y_chained

In [4]:
y_real = take_y_reals()


folder_path = "data/forecast_results"
forecast_yoy_all_cat = concatenate_csv_files(folder_path)
date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')

In [5]:
# take the weights:
weight_f = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_food.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_f = weight_f.iloc[1, :].to_frame().reset_index()
weight_f.columns = weight_f.iloc[0]
weight_f = weight_f[1:]
weight_f.rename(columns={"TIME": "date", "Germany": 'weight_f'}, inplace=True)

weight_e = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_energy.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_e = weight_e.iloc[1, :].to_frame().reset_index()
weight_e.columns = weight_e.iloc[0]
weight_e = weight_e[1:]
weight_e.rename(columns={"TIME": "date", "Germany": 'weight_e'}, inplace=True)

weight_n = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_neig.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_n = weight_n.iloc[1, :].to_frame().reset_index()
weight_n.columns = weight_n.iloc[0]
weight_n = weight_n[1:]
weight_n.rename(columns={"TIME": "date", "Germany": 'weight_n'}, inplace=True)

weight_s = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_services.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_s = weight_s.iloc[1, :].to_frame().reset_index()
weight_s.columns = weight_s.iloc[0]
weight_s = weight_s[1:]
weight_s.rename(columns={"TIME": "date", "Germany": 'weight_s'}, inplace=True)

weights = pd.concat([weight_f, weight_e, weight_n, weight_s], axis=1)
weights = weights.loc[:, ~weights.columns.duplicated()]
weights['date'] = weights['date'].astype(int)
for col in weights.columns[1:]:
    weights[col] = weights[col].astype(float)

weights.loc[:, 'total'] = weights.iloc[:, 1:5].sum(axis=1)


/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's defaul

In [6]:
models = ['ar_1','prc','xgb','ridge', 'lasso', 'cb_ridge_pcr', 'cb_ridge_prc_lasso', 'cb_ridge_lasso','xgb_pca']

horizons = [1, 2, 3]

for model in models:
    print(model)
    for h in horizons:
        # get the categorical data:
        new_df = extract_forecast_model_h(forecast_yoy_all_cat, model, h)
        new_df.set_index(date_range, inplace=True)

        assert new_df.shape[1] == 4

        # transform:
        food_chained = chain.transform_back_chained(new_df.iloc[:, 0], y_real.iloc[:, 0])
        food_unchain = chain.unchain_series(food_chained, y_real.iloc[:, 0])
        energy_chained = chain.transform_back_chained(new_df.iloc[:, 1], y_real.iloc[:, 1])
        energy_unchain = chain.unchain_series(energy_chained, y_real.iloc[:, 1])
        neig_chained = chain.transform_back_chained(new_df.iloc[:, 2], y_real.iloc[:, 2])
        neig_unchained = chain.unchain_series(neig_chained, y_real.iloc[:, 2])
        services_chained = chain.transform_back_chained(new_df.iloc[:, 3], y_real.iloc[:, 3])
        services_unchained = chain.unchain_series(services_chained, y_real.iloc[:, 3])

        # take the products of respective category and weights
        food_unchain.name = 'food_unchain'
        food_unchain = food_unchain.to_frame()
        food_unchain.loc[:, 'year'] = food_unchain.index.year
        food_unchain
        merged_df = pd.merge(food_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_food = merged_df.loc[:, 'food_unchain'].mul(merged_df.loc[:, 'weight_f'])

        energy_unchain.name = 'e_unchain'
        e_unchain = energy_unchain.to_frame()
        e_unchain.loc[:, 'year'] = e_unchain.index.year
        merged_df = pd.merge(e_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_e = merged_df.loc[:, 'e_unchain'].mul(merged_df.loc[:, 'weight_e'])


        neig_unchained.name = 'n_unchain'
        n_unchain = neig_unchained.to_frame()
        n_unchain.loc[:, 'year'] = n_unchain.index.year
        merged_df = pd.merge(n_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_n = merged_df.loc[:, 'n_unchain'].mul(merged_df.loc[:, 'weight_n'])

        services_unchained.name = 's_unchain'
        s_unchain = services_unchained.to_frame()
        s_unchain.loc[:, 'year'] = s_unchain.index.year
        merged_df = pd.merge(s_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_s = merged_df.loc[:, 's_unchain'].mul(merged_df.loc[:, 'weight_s'])

        # take the sum of all product:
        mul_sum = pd.concat([mul_food, mul_e, mul_n, mul_s], axis=1)
        mul_sum.loc[:, 'hicp_unchained'] = mul_sum.sum(axis=1)/1000

        date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')
        mul_sum.set_index(date_range, inplace=True)

        # chain the results:
        y_real_hicp = mine_g.load_excel('data/hicp_all.xlsx', 'hicp_all', verbose=0)
        hicp_chained_final = transform_to_chained(mul_sum.loc[:, 'hicp_unchained'], y_real_hicp.iloc[:, 0])

        y_real_hicp.loc[:, 'last_y'] = y_real_hicp.iloc[:, 0].shift(12)
        y_real_hicp = y_real_hicp[(y_real_hicp.index > mine_g.train_test_split_date) & (y_real_hicp.index <= mine_g.max_X_date)]

        hicp_all_yoy = (hicp_chained_final/y_real_hicp.loc[:, 'last_y']-1)*100
        hicp_all_yoy.name = f'{model}_h_{h}'   
        forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')
        new_forecast = pd.concat([forecast, hicp_all_yoy.reset_index(drop=True)], axis=1)

        new_forecast.to_csv('data/headline_forecast/head_infl_forecast.csv', index=False)


ar_1
prc
xgb
ridge
lasso
cb_ridge_pcr
cb_ridge_prc_lasso
cb_ridge_lasso
xgb_pca


In [7]:
new_forecast

,ar_110_h_1,ar_110_h_2,ar_110_h_3,ar_ols_h_1,ar_ols_h_2,ar_ols_h_3,ar_1_h_1,ar_1_h_2,ar_1_h_3,prc_h_1,...,cb_ridge_pcr_h_3,cb_ridge_prc_lasso_h_1,cb_ridge_prc_lasso_h_2,cb_ridge_prc_lasso_h_3,cb_ridge_lasso_h_1,cb_ridge_lasso_h_2,cb_ridge_lasso_h_3,xgb_pca_h_1,xgb_pca_h_2,xgb_pca_h_3
0,0.201236,0.329779,1.160298,0.305073,0.400090,1.253522,0.383403,0.416460,1.208244,0.434589,...,0.483564,0.716691,0.686040,0.797024,0.857743,0.620477,0.660402,0.561827,0.871591,0.403591
1,0.377993,0.201232,0.333175,0.493679,0.399921,0.486686,0.466180,0.422269,0.455260,0.157296,...,0.558318,0.468606,0.626231,0.661938,0.624261,0.800519,0.701670,1.082021,0.754384,0.268074
2,-0.025772,0.382279,0.201233,0.029098,0.572149,0.486532,-0.025176,0.477711,0.441547,-0.310435,...,0.561244,-0.059933,0.282401,0.486276,0.065318,0.419587,0.676952,0.158254,0.731692,0.318251
3,0.070035,-0.036898,0.381662,0.212929,0.147909,0.643806,0.246660,0.139806,0.619105,-0.202756,...,0.282833,-0.025881,-0.080610,0.296273,0.062557,0.068927,0.470845,0.166071,0.312875,0.504312
4,-0.238173,0.074585,-0.035253,-0.152066,0.315777,0.256404,-0.158952,0.255937,0.160104,-0.258924,...,-0.212678,-0.267924,-0.169376,-0.152999,-0.272424,-0.132821,-0.049843,0.229921,0.219093,0.160377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,8.453284,8.295415,8.659386,7.862867,7.111033,6.988278,8.025836,7.474440,7.310120,8.816317,...,10.419346,9.112885,9.472832,9.697823,9.261169,9.776371,9.971011,7.904274,6.013555,6.759917
80,8.796783,8.455887,8.295984,8.188482,7.301452,6.614902,8.257302,7.499488,6.986108,9.622693,...,10.276790,9.557401,9.755540,9.619639,9.524754,9.927692,9.855542,7.765193,6.239154,6.838412
81,10.712508,8.800719,8.455609,10.074116,7.598793,6.788786,10.387207,7.982072,7.265374,11.501768,...,11.903909,11.743405,11.241041,10.954795,11.864223,11.600653,11.420544,8.687738,7.266513,7.699012
82,11.523992,10.729384,8.800305,10.680094,9.320693,7.060308,10.740210,9.761903,7.482193,12.050228,...,12.854960,11.933305,12.343196,12.100559,11.874844,12.456184,12.622823,9.377199,7.865419,7.953460


----

# Compare OLS for headline and OLS from individual cat aggregated

In [8]:
forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')

real_hicp_yoy = pd.read_csv('data/preprocessed/head_inflation.csv', parse_dates = True, index_col='date')
real_hicp_yoy_test = real_hicp_yoy[(real_hicp_yoy.index > mine_g.train_test_split_date) & (real_hicp_yoy.index <= mine_g.max_X_date)]


In [11]:
rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast[h_cols]
    
    for col in [ col for col in forecast_df_h.columns if "ar_ols" not in col]:
        print(col)
        print(dm_test(real_hicp_yoy_test.iloc[:, 0], forecast_df_h[col], forecast_df_h.loc[:, f'ar_ols_h_{h}'], h = h, harvey_correction=True))


    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], real_hicp_yoy_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]
rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, 11), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])


ar_110_h_1
(-1.6769896095002441, 0.0973070531150434)
ar_1_h_1
(-2.509296608379326, 0.014041352791209554)
prc_h_1
(-1.4256522443636404, 0.1577188484032771)
xgb_h_1
(3.8123072933860644, 0.00026353024678415)
ridge_h_1
(-1.3968662707348651, 0.16617756166952335)
lasso_h_1
(-1.1413650878717199, 0.2570003729897524)
cb_ridge_pcr_h_1
(-1.3968662707348651, 0.16617756166952335)
cb_ridge_prc_lasso_h_1
(-1.7739972942888729, 0.0797310027548146)
cb_ridge_lasso_h_1
(-1.5297925317532695, 0.12987056895192084)
xgb_pca_h_1
(3.107179116078062, 0.0025859367370374906)
ar_110_h_2
(-1.5284227334846807, 0.13020984156895388)
ar_1_h_2
(-1.750981973746627, 0.08364361496407262)
prc_h_2
(-1.1162976528397033, 0.2675160514289605)
xgb_h_2
(2.4849772600907745, 0.014964546962963033)
ridge_h_2
(-1.4633495035463375, 0.14714862078269397)
lasso_h_2
(-1.3160512355498044, 0.19178002248812587)
cb_ridge_pcr_h_2
(-1.4633495035463375, 0.14714862078269397)
cb_ridge_prc_lasso_h_2
(-1.4572585234459576, 0.148818158199998)
cb_ridge_las

In [12]:
rmse

,ar_110,ar_ols,ar_1,prc,xgb,ridge,lasso,cb_ridge_pcr,cb_ridge_prc_lasso,cb_ridge_lasso,xgb_pca
h_1,0.626385,0.696370,0.666286,0.595058,1.108621,0.604577,0.625565,0.604577,0.584246,0.601347,0.980802
h_2,0.883125,1.075871,1.013742,0.865649,1.299482,0.756373,0.819226,0.756373,0.780401,0.766179,1.427536
h_3,1.116769,1.431566,1.337457,1.080243,1.190141,0.928331,1.012402,0.928331,0.974572,0.952910,1.324265


In [ ]:
rmse.to_csv("data/report_rmse/total.csv")

In [13]:
rmse_comparative = [round(rmse[col]/rmse[f'ar_ols'], 2) for col in rmse.columns]
rmse_comparative

[h_1    0.90
 h_2    0.82
 h_3    0.78
 dtype: float64,
 h_1    1.0
 h_2    1.0
 h_3    1.0
 Name: ar_ols, dtype: float64,
 h_1    0.96
 h_2    0.94
 h_3    0.93
 dtype: float64,
 h_1    0.85
 h_2    0.80
 h_3    0.75
 dtype: float64,
 h_1    1.59
 h_2    1.21
 h_3    0.83
 dtype: float64,
 h_1    0.87
 h_2    0.70
 h_3    0.65
 dtype: float64,
 h_1    0.90
 h_2    0.76
 h_3    0.71
 dtype: float64,
 h_1    0.87
 h_2    0.70
 h_3    0.65
 dtype: float64,
 h_1    0.84
 h_2    0.73
 h_3    0.68
 dtype: float64,
 h_1    0.86
 h_2    0.71
 h_3    0.67
 dtype: float64,
 h_1    1.41
 h_2    1.33
 h_3    0.93
 dtype: float64]